Para o Trabalho prático eu segui 
[este tutorial](https://raw.githubusercontent.com/ssloy/tinyrenderer/master/obj/diablo3_pose/diablo3_pose.obj)
colocando enfâse na parte voltada para Álgebra liner

In [36]:
!python --version

Python 3.7.15


In [1]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import requests

from collections import namedtuple

Definindo algumas constantes

In [2]:
WIDTH = 512
HEIGHT = 512

def color(r: int, g: int, b: int) -> tuple:
  return b, g, r

Importando arquivos do github

In [3]:
DATA = {
  "african_head.obj": requests.get("https://raw.githubusercontent.com/ssloy/tinyrenderer/master/obj/african_head/african_head.obj"),
  "african_head_diffuse.tga": requests.get("https://raw.githubusercontent.com/ssloy/tinyrenderer/master/obj/african_head/african_head_diffuse.tga"),
  "diablo3_pose.obj": requests.get("https://raw.githubusercontent.com/ssloy/tinyrenderer/master/obj/diablo3_pose/diablo3_pose.obj"),
  "diablo3_pose_diffuse.tga": requests.get("https://raw.githubusercontent.com/ssloy/tinyrenderer/master/obj/diablo3_pose/diablo3_pose_diffuse.tga"),
}

Criando função para interpretar o arquivo TGA

TODO linkar um site com a especificação do TGA

In [33]:
def read_tga(tga: bytearray) -> np.array:
  tga = tga[::-1] # Com a lista invertida fazer pop fica mais eficiente

  def read_bytes(tga, length: int = 1) -> list:
    return [tga.pop() for _ in range(length)]

  def read_int(tga, length: int = 1) -> int:
    return int.from_bytes([tga.pop() for _ in range(length)], "little")

  def skip_bytes(tga, length: int = 1) -> None:
    for _ in range(length):
      tga.pop()


  # > HEADER
  # ID length
  image_id_length = read_int(tga)

  # Color map type
  color_map = read_int(tga)
  assert color_map == 0, f"{color_map=} não implementado!"

  # image type (Compression and color types)
  image_type = read_int(tga)
  assert image_type in [2, 10], f"{image_type=} não implementado!"
  rle = image_type == 10

  # >> Color Map specification
  # Ignored because color map type is 0
  skip_bytes(tga, 5)

  # >> Image specification
  # X and Y origin
  origin = (read_int(tga, 2), read_int(tga, 2))
  assert origin == (0, 0), f"{origin=} não implementado! Precisa ser (0, 0)!"

  # Image width and height
  width, height = read_int(tga, 2), read_int(tga, 2)
  
  # Pixel Depth
  color_depth = read_int()
  assert color_depth == 24, f"{color_depth=} não implementado!"

  image_descriptor = read_int()
  assert (image_descriptor & 0b1100111) == 0, f"{image_descriptor=} não implementado!"
  flip_horizontally = (image_descriptor & 0b0001000) != 0;
  flip_vertically = (image_descriptor & 0b0010000) != 0;

  # > image and color map data
  # Image ID
  skip_bytes(tga, image_id_length)
  # Color Map (Asserted to no color map)
  # Image data
  image = np.zeros((width * height, 3), np.uint8)
  if rle:
    length = 0
    while length > width * height:
      packet_size = read_int()
      if packet_size & 0b10000000 == 0:
        # Raw packet
        for _ in range(packet_size + 1):
          image[length] = read_bytes(tga, 3)
          length += 1
      else:
        # RLE packet
        packet_size = packet_size & 0b01111111
        color = read_bytes(tga, 3)
        for _ in range(packet_size + 1):
          image[length] = color
          length += 1
  else:
    for _ in range(width * height):
      image[length] = read_bytes(tga, 3)
      length += 1
  image = image.reshape(width, height, 3)

  # if flip_horizontally:
  #   image = image.flip(0)
  # if flip_vertically:
  #   image = image.flip(1)
  return image

SyntaxError: ignored

In [25]:
img = read_tga(bytearray(DATA["african_head_diffuse.tga"].content))
# cv2_imshow(img)
img = read_tga(bytearray(DATA["diablo3_pose_diffuse.tga"].content))
# cv2_imshow(img)

2801513
['\x87', '7', 'E', 'X', '\x81', '9', 'G', 'Z', '\x83', '8', 'F', 'Y', '\x81', '7', 'E', 'X', '\x87', '7', 'D', 'Z', '\x00', '6', 'D', 'W', '\x84', '7', 'E', 'X', '\x81', '8', 'F', 'Y', '\x00', '6', 'B', 'Z', '\x83', '5', 'A', 'Y', '\x00', '6', 'B', 'Z', '\x81', '7', 'C', '[', '\x83', '7']
2970976
['\x03', '\x0c', '\x11', '\x1e', '\x0b', '\x10', '\x1d', '\t', '\x0f', '\x1e', '\x07', '\r', '\x1d', '\x83', '\x07', '\r', '\x1c', '\x87', '\x08', '\x0e', '\x1d', '\x8d', '\t', '\x0f', '\x1e', '\x82', '\t', '\x0e', '\x1d', '\x82', '\t', '\x0e', '\x1c', '\x00', '\t', '\r', '\x1b', '\x82', '\x08', '\r', '\x1b', '\x82', '\x07', '\x0c', '\x1a', '\x00', '\x07', '\x0b', '\x19', '\x81']


In [6]:
img = np.zeros((WIDTH, HEIGHT, 3), np.uint8)

for i in range(10):
  for j in range(10):
    img[i,j] = color(255,0,0)
img = cv.flip(img, 0)
cv2_imshow(img)

NameError: ignored